### Обучение TextToLabel

In [261]:
import pandas as pd

files: list[str] = [
    "dataset/transcription/hr_bot_clear.csv",
    "dataset/transcription/hr_bot_noise.csv",
    "dataset/transcription/hr_bot_synt.csv",
    "dataset/transcription/luga.csv",
]

li = []
for filename in files:
    df = pd.read_csv(filename)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df = df.sample(frac=1)
df.head()
df = df.groupby('label').apply(lambda x: x.sample(n=min(len(x), 120), random_state=42)).reset_index(drop=True)
df.value_counts("label")

/tmp/ipykernel_54038/2702262179.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('label').apply(lambda x: x.sample(n=min(len(x), 120), random_state=42)).reset_index(drop=True)


label
18    120
4     120
6     120
8     120
10    120
12    120
13    120
20    120
17    119
1     117
7     116
19    115
2     112
15    111
5     110
16    103
9     102
3     102
0     100
11    100
14     79
21     18
22     11
Name: count, dtype: int64

In [262]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    HashingVectorizer,
    CountVectorizer,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC

from sklearn.pipeline import make_pipeline

# vectorizer = HashingVectorizer(
#     lowercase=True, n_features=2**10, stop_words="english"
# )
# vectorizer = TfidfVectorizer(
#     lowercase=True, sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english", ngram_range=(1, 2)
# )
vectorizer = CountVectorizer(ngram_range=(1, 2))

# classifier = RandomForestClassifier(n_estimators=200, random_state=42)
# classifier = LogisticRegression()
# classifier = KNeighborsClassifier(n_neighbors=5)
# classifier = MultinomialNB()
classifier = LinearSVC(max_iter=20000, dual = True)
# classifier = AdaBoostClassifier()
# classifier = GradientBoostingClassifier()

pipeline = make_pipeline(vectorizer, classifier)

In [263]:
X = df["transcription"]
y = df["label"]

In [264]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('linearsvc', LinearSVC(dual=True, max_iter=20000))])

In [265]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

y_pred = pipeline.predict(X_test)
f1_weighted = f1_score(y_test, y_pred, average='weighted')
print(f"F1: {f1_weighted:.2f}")

F1: 0.67


In [266]:
import joblib
joblib.dump(pipeline, "trained/text2label.pkl")

['trained/text2label.pkl']